In [1]:
import pandas as pd
import pybaseball as pyb
from IPython.display import HTML
import dataframe_image as dfi
import seaborn as sns

#ian anderson
player_name = 'anderson ian'

player_id_table = pyb.playerid_lookup(*player_name.split())

player_id = player_id_table.values[0][2]

pitches = pyb.statcast_pitcher(start_dt='2020-07-23', end_dt='2020-10-28', player_id=player_id)

pitches = pitches[['balls', 'strikes', 'pitch_name']]

pitches['balls'] = pitches['balls'].astype(str)
pitches['strikes'] = pitches['strikes'].astype(str)

pitches['count'] = pitches['balls'] + '-' + pitches['strikes']
pitches['pitch'] = 1

pitches = pitches[['pitch', 'count', 'pitch_name']]

pitches = pitches.pivot_table(columns='pitch_name', index='count', aggfunc='count').fillna(0)

pitches['pitch_name', 'total_thrown'] = pitches.sum(axis=1)

pitches.columns = pitches.columns.droplevel(level=0)

for pitch_name in pitches.columns.tolist()[:-1]:
    pitches[pitch_name] = pitches[pitch_name] / pitches['total_thrown']

pitches.drop('total_thrown', axis=1, inplace=True)

player_name_actual = player_name.split()[-1].capitalize() + ' ' + player_name.split()[0].capitalize()
df_title = f"\n{player_name_actual}'s pitch selection per count\n"

formatters = {}
for pitch in pitches.columns.tolist():
    formatters[pitch] = '{0:.0%}'

pitches_styled = pitches.style.background_gradient(cmap=sns.light_palette('seagreen', as_cmap=True)).format(formatters)

dfi.export(pitches_styled, f'img/{player_name}.png')

HTML(pitches_styled.render(table_title=df_title))

Error: Pip module Unable to parse debugpy output, please log an issue with https://github.com/microsoft/vscode-jupyter is required for debugging cells. You will need to install it to debug cells.